In [ ]:
#@title
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

#Hyperparameters
NUM_CLASSES=4
NUM_FEATURES=2
RANDOM_SEED=42
LR=0.1              #learning rate

#create multiclass data
X_blob, y_blob=make_blobs(n_samples=1000, n_features=NUM_FEATURES, centers=NUM_CLASSES, cluster_std=1.5, random_state=RANDOM_SEED)
#X_blob[:5], y_blob[:5]
#plt.scatter(x=X_blob[:, 0], y=X_blob[:, 1], c=y_blob, cmap=plt.cm.brg)

#Turn data into tensor
X_blob=torch.from_numpy(X_blob).type(torch.float32)
y_blob=torch.from_numpy(y_blob).type(torch.LongTensor) #LongTensor

#split into training and test
X_blob_train, X_blob_test, y_blob_train, y_blob_test=train_test_split(X_blob, y_blob, test_size=0.2, random_state=RANDOM_SEED)
plt.scatter(x=X_blob[:, 0], y=X_blob[:, 1], c=y_blob, cmap=plt.cm.brg)

In [70]:
#Device agnostic code
device='cuda' if torch.cuda.is_available() else 'cpu'

#Building a multiclass classification model in PyTorch
class BlobModel(nn.Module):
  def __init__(self, input_features, output_features, hidden_units=8):
    super().__init__()
    self.layer_1=nn.Linear(in_features=input_features, out_features=hidden_units)
    self.layer_2=nn.Linear(in_features=hidden_units, out_features=hidden_units)
    self.layer_3=nn.Linear(in_features=hidden_units, out_features=output_features)
    #self.relu=nn.ReLU()

  def forward(self,x):
    #return self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x))))) # x -> layer 1 -> layer 2 -> y
    return self.layer_3(self.layer_2(self.layer_1(x))) # x -> layer 1 -> layer 2 -> y

#Instantiate a model and send to device
model_0=BlobModel(input_features=NUM_FEATURES, output_features=NUM_CLASSES).to(device)
#model_0

In [71]:
#Create a loss function for multiclass classification
loss_fn=nn.CrossEntropyLoss()

#create an optimizer
optimizer=torch.optim.SGD(params=model_0.parameters(), lr=LR)

def accuracy_fn(y_true, y_pred):
  correct=torch.eq(y_true, y_pred).sum().item()
  acc=(correct/len(y_pred))*100
  return acc

#Initial evaluation: not really needed
'''
model_0.eval()
with torch.inference_mode():
  y_logits=model_0(X_blob_test.to(device))

#logits (raw output of the model) -> prediction probility -> prediction labes
y_pred_probs=torch.softmax(y_logits, dim=1)
y_preds=torch.argmax(y_pred_probs, dim=1)
y_preds[:10]
'''

'\nmodel_0.eval()\nwith torch.inference_mode():\n  y_logits=model_0(X_blob_test.to(device))\n\n#logits (raw output of the model) -> prediction probility -> prediction labes\ny_pred_probs=torch.softmax(y_logits, dim=1)\ny_preds=torch.argmax(y_pred_probs, dim=1)\ny_preds[:10]\n'

In [72]:
#Create a training loop
torch.manual_seed(42)
torch.cuda.manual_seed(42)

epochs=100
X_blob_train, y_blob_train =X_blob_train.to(device), y_blob_train.to(device)
X_blob_test, y_blob_test =X_blob_test.to(device), y_blob_test.to(device)

for epoch in range(epochs):
  #Training
  model_0.train()
  y_logits=model_0(X_blob_train)
  y_pred=torch.softmax(y_logits, dim=1).argmax(dim=1)
  loss=loss_fn(y_logits, y_blob_train)
  acc=accuracy_fn(y_true=y_blob_train, y_pred=y_pred)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  #Testing
  model_0.eval()
  with torch.inference_mode():
    test_logits=model_0(X_blob_test)
    test_preds=torch.softmax(test_logits, dim=1).argmax(dim=1)

    test_loss=loss_fn(test_logits, y_blob_test)
    test_acc=accuracy_fn(y_true=y_blob_test, y_pred=test_preds)

  #print out what is happening
  if epoch % 10 ==0:
    print(f'Epoch: {epoch} | Train loss: {loss:.4f}, Train acc: {acc:.2f}% | Test loss: {test_loss: .4f}, Test acc: {test_acc:.2f}%')


Epoch: 0 | Train loss: 1.0432, Train acc: 65.50% | Test loss:  0.5786, Test acc: 95.50%
Epoch: 10 | Train loss: 0.1440, Train acc: 99.12% | Test loss:  0.1304, Test acc: 99.00%
Epoch: 20 | Train loss: 0.0806, Train acc: 99.12% | Test loss:  0.0722, Test acc: 99.50%
Epoch: 30 | Train loss: 0.0592, Train acc: 99.12% | Test loss:  0.0513, Test acc: 99.50%
Epoch: 40 | Train loss: 0.0489, Train acc: 99.00% | Test loss:  0.0410, Test acc: 99.50%
Epoch: 50 | Train loss: 0.0429, Train acc: 99.00% | Test loss:  0.0349, Test acc: 99.50%
Epoch: 60 | Train loss: 0.0391, Train acc: 99.00% | Test loss:  0.0308, Test acc: 99.50%
Epoch: 70 | Train loss: 0.0364, Train acc: 99.00% | Test loss:  0.0280, Test acc: 99.50%
Epoch: 80 | Train loss: 0.0345, Train acc: 99.00% | Test loss:  0.0259, Test acc: 99.50%
Epoch: 90 | Train loss: 0.0330, Train acc: 99.12% | Test loss:  0.0242, Test acc: 99.50%


In [73]:
#Now the model is good, make prediction
model_0.eval()
with torch.inference_mode():
  y_logits=model_0(X_blob_test)
  y_preds=torch.argmax(torch.softmax(y_logits, dim=1), dim=1)

print('Prediction:', y_preds[:10], '\nOriginal:', y_blob_test[:10])



Prediction: tensor([1, 3, 2, 1, 0, 3, 2, 0, 2, 0]) 
Original: tensor([1, 3, 2, 1, 0, 3, 2, 0, 2, 0])
